In [25]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import sys
import pandas as pd
import numpy as np
import cohorts
from cohorts.functions import missense_snv_count
import query_tcga
#from query_tcga import cohort, config, cache
#from query_tcga import query_tcga as qtcga
import logging
logging.basicConfig(level=logging.DEBUG)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Creating a Cohort

There are several ways to create a Cohort using [cohorts](http://github.com/hammerlab/cohorts).

In our first example, we [created a Cohort from clinical data](http://github.com/jburos/tcga-blca/blob/master/Example%20-%20Creating%20a%20cohort%20from%20clinical%20data%20using%20python2.ipynb). We also showed several of the data summary methods available, using clinical covariates as exampes. 

But the true value of Cohorts lies in its ability to summarize various categories of genetic & molecular data and to incorporate those data into the analysis.

For example, we are going to again use `TCGA` data from the `BLCA` cohort (generated using the `get_clinical_data.py` command-line script available here). 


## Part 2: Creating a Cohort with somatic mutations

### download VCF files

First, we will set up the values for the `query_tcga` package to download the VCF files.

In [26]:
query_tcga.config.set_value(USE_CACHE=True)
query_tcga.config.set_value(GDC_TOKEN_PATH='/Users/jacquelineburos/Downloads/gdc-user-token.2016-09-26T12-23-27-04-00.txt')
query_tcga.config.set_value(GDC_DATA_DIR='/Users/jacquelineburos/projects/tcga-blca/data/gdc/')
query_tcga.config.set_value(DEFAULT_SIZE=10)
query_tcga.cache.setup_cache()

We will use `query_tcga` to download VCF files for 30 patients in this `TCGA` project.

In [27]:
vcf_files = query_tcga.samples.download_vcf_files(project_name='TCGA-BLCA', n=30)

This function returns a list of files, plus a `.fileinfo` attribute summarizing file contents.

In [54]:
vcf_files.fileinfo.head(n=1)

,file_id,filepath,reference_name,analysis_id,case_id,data_category,data_type,file_name,samples,submitter_id
0,04cfe9cc-e74d-4f67-9652-eb9fece22e89,data/gdc/04cfe9cc-e74d-4f67-9652-eb9fece22e89/...,GRCh38,ab2ef33d-678f-43f3-95b0-f3c02ae28025,f953dc5d-58bf-4936-b9fe-74b45e8c00e5,Simple Nucleotide Variation,Raw Simple Somatic Mutation,04cfe9cc-e74d-4f67-9652-eb9fece22e89.vcf.gz,"[{u'submitter_id': u'TCGA-DK-A3IV-01A', u'tumo...",TCGA-DK-A3IV


In [65]:
vcf_fileinfo = vcf_files.fileinfo.loc[:,['submitter_id','filepath']]
vcf_fileinfo.rename(columns = {'filepath': 'snv_vcf_paths'}, inplace=True)
vcf_fileinfo['patient_id'] = vcf_fileinfo['submitter_id'].apply(lambda x: x.split('-')[2])
vcf_fileinfo.head()

,submitter_id,snv_vcf_paths,patient_id
0,TCGA-DK-A3IV,data/gdc/04cfe9cc-e74d-4f67-9652-eb9fece22e89/...,A3IV
1,TCGA-GU-A42P,data/gdc/0e4ff65d-ce0c-4494-a7ab-d4fb39689bf4/...,A42P
2,TCGA-XF-AAMH,data/gdc/06f5a911-6cab-4b31-8daa-86830972b1c5/...,AAMH
3,TCGA-GC-A3I6,data/gdc/0cadfaa3-70b5-4874-9130-c6757e684329/...,A3I6
4,TCGA-DK-AA6Q,data/gdc/06b001a4-7660-4ae9-9a77-03d61dfb58b1/...,AA6Q


### load clinical data 

In [111]:
clinical_data = pd.read_csv('data/clinical.csv', sep='|')
clinical_data = clinical_data.merge(vcf_fileinfo, on='patient_id', how='left')
assert clinical_data['snv_vcf_paths'].count()>0
clinical_data.dropna(subset=['snv_vcf_paths'], inplace=True, axis=0)
#clinical_data.head()

In [123]:
def prep_patient_data(row):
    # capture key outcome data elements
    patient_id = row['patient_id']
    deceased = row['vital_status'] != 'Alive'
    progressed = row['treatment_outcome_at_tcga_followup'] != 'Complete Response'
    censor_time = float(row['last_contact_days_to'])
    deceased_time = float(row['death_days_to'])
    progressed_time = float(row['new_tumor_event_dx_days_to'])
    
    # compute age at diagnosis
    row['age'] = (-1*row['birth_days_to'])/365.25
    
    # save back in 'row' as-is so that we can see raw values
    row['progressed_time'] = progressed_time
    row['deceased_time'] = deceased_time
    row['censor_time'] = censor_time
    row['progressed'] = progressed
    row['deceased'] = deceased

    # clean up censor time - a number of obs have NaN values 
    if np.isnan(censor_time):
        censor_time = max(progressed_time, deceased_time, censor_time)
    if censor_time > progressed_time:
        censor_time = progressed_time
    if censor_time > deceased_time:
        censor_time = deceased_time

    # save time-to-event-or-censor data elements
    os = deceased_time if deceased else censor_time
    pfs = progressed_time if progressed else os
    
    # again, make sure outcomes aren't NaN
    if np.isnan(os):
        os = censor_time

    if np.isnan(pfs):
        pfs = os
    
    # save transformed versions of outcome back to 'row' object for inspection
    row['pfs'] = pfs
    row['os'] = os
    row['censor_time'] = censor_time
    
    # force progressed time to be < os 
    pfs = min(pfs, os) 
    
    # definition of benefit for this cohort
    benefit = pfs <= 365.25
    
    # these conditions are required by Cohorts
    assert(not np.isnan(pfs))
    assert(not np.isnan(os))
    assert pfs <= os, 'PFS {pfs} is not <= OS {os} for Patient {patid}'.format(pfs=pfs, os=os, patid=patient_id)
    
    # capture snv_vcf_paths, if they exist
    if 'snv_vcf_paths' in row.keys() and isinstance(row['snv_vcf_paths'], str):
        snv_vcf_paths = list(row['snv_vcf_paths'])
    else:
        snv_vcf_paths = None
    
    # create our patient object
    patient = cohorts.Patient(
        id=str(patient_id),
        deceased=deceased,
        progressed=progressed,
        os=os,
        pfs=pfs,
        benefit=benefit,
        additional_data=row,
        snv_vcf_paths=snv_vcf_paths,
    )
    return(patient)


As in the earlier example, we will process the rows of our clinical data into a list of Patients

In [124]:
patients = []
for (i, row) in clinical_data.iterrows():
    patients.append(prep_patient_data(row))

And, create a Cohort object from the list

In [127]:
blca_cohort2 = cohorts.Cohort(patients, cache_dir='data-cache2')

TypeError: __init__() got an unexpected keyword argument 'on'

## retrieve dataframe for a cohort

In [126]:
df = blca_cohort2.as_dataframe(on=[missense_snv_count])

Variants did not exist for patient A42C
Variants did not exist for patient A40E
Variants did not exist for patient A541
Variants did not exist for patient A7PW
Variants did not exist for patient AAN0
Variants did not exist for patient AAMT
Variants did not exist for patient AAN4
Variants did not exist for patient A6TH
Variants did not exist for patient AAMH
Variants did not exist for patient A2LA
Variants did not exist for patient A3MH
Variants did not exist for patient AAML
Variants did not exist for patient A3WX
Variants did not exist for patient AA6Q
Variants did not exist for patient A0S7
Variants did not exist for patient A2OE
Variants did not exist for patient A5KF
Variants did not exist for patient A3IV
Variants did not exist for patient AA76
Variants did not exist for patient A3I6
Variants did not exist for patient A6B5
Variants did not exist for patient AA26
Variants did not exist for patient A1AB
Variants did not exist for patient A9TC
Variants did not exist for patient A2PC


In [117]:
df = blca_cohort2.as_dataframe([missense_snv_count])
df.describe()

Variants did not exist for patient 0    b9808ef9-e917-45f9-9e0f-b86f7834b5ff
Name: case_id, dtype: object
Variants did not exist for patient 0    aa6142af-bad7-493c-98a0-fd070de39073
Name: case_id, dtype: object
Variants did not exist for patient 0    d51abc3a-a652-4b13-9a41-ce52be87218b
Name: case_id, dtype: object
Variants did not exist for patient 0    045fe498-6a07-41ff-9956-dc0fb0483e7f
Name: case_id, dtype: object
Variants did not exist for patient 0    e7e95597-ef88-4442-8c01-8d633822e7b5
Name: case_id, dtype: object
Variants did not exist for patient 0    ada95115-275f-4229-aa00-f298a5ff931c
Name: case_id, dtype: object
Variants did not exist for patient 0    b6c36037-56b1-4957-8df8-fda72b0d3c39
Name: case_id, dtype: object
Variants did not exist for patient 0    08d13748-0946-49a1-8eb3-5afd1deebc84
Name: case_id, dtype: object
Variants did not exist for patient 0    0ec37c2c-8d90-48c1-9cbc-69fe6473c980
Name: case_id, dtype: object
Variants did not exist for patient 0    797a71

AttributeError: 'tuple' object has no attribute 'describe'

## plot survival

In [ ]:
blca_cohort.plot_survival(on='tobacco_smoking_history_indicator')

In [ ]:
blca_cohort.plot_survival(on='tobacco_smoking_history_indicator', threshold=1)

In [ ]:
blca_cohort.plot_survival(on='tobacco_smoking_history_indicator', threshold=1, how='pfs')

You can also pass a function to the `plot_survival` method. 

For example, let's say we wanted to summarized survival according to the tumor grade.

First, we create a function to classify our tumor type:


In [ ]:
def high_grade_tumor(row):
    return row['tumor_grade'] == 'High Grade'

We then pass this function to the `plot_survival` method. 

(Behind the scenes, the function will be applied to each `row` of our dataframe & the result will be summarized.)

In [ ]:
blca_cohort.plot_survival(on=high_grade_tumor, how='pfs')

## Summarizing somatic mutation burden

In [116]:
blca_cohort2.plot_survival(missense_snv_count)

Variants did not exist for patient 0    b9808ef9-e917-45f9-9e0f-b86f7834b5ff
Name: case_id, dtype: object
Variants did not exist for patient 0    aa6142af-bad7-493c-98a0-fd070de39073
Name: case_id, dtype: object
Variants did not exist for patient 0    d51abc3a-a652-4b13-9a41-ce52be87218b
Name: case_id, dtype: object
Variants did not exist for patient 0    045fe498-6a07-41ff-9956-dc0fb0483e7f
Name: case_id, dtype: object
Variants did not exist for patient 0    e7e95597-ef88-4442-8c01-8d633822e7b5
Name: case_id, dtype: object
Variants did not exist for patient 0    ada95115-275f-4229-aa00-f298a5ff931c
Name: case_id, dtype: object
Variants did not exist for patient 0    b6c36037-56b1-4957-8df8-fda72b0d3c39
Name: case_id, dtype: object
Variants did not exist for patient 0    08d13748-0946-49a1-8eb3-5afd1deebc84
Name: case_id, dtype: object
Variants did not exist for patient 0    0ec37c2c-8d90-48c1-9cbc-69fe6473c980
Name: case_id, dtype: object
Variants did not exist for patient 0    797a71

IndexError: list index out of range